# 🛒 E-Commerce Analytics ETL using FakeStore API

This notebook builds an end-to-end ETL (Extract, Transform, Load) pipeline using the FakeStore API.  
The goal is to transform raw e-commerce API data into an analysis-ready dataset and derive key business metrics such as revenue and category performance.

In [10]:
import pandas as pd

## 🔹 Extract

In this step, we fetch raw data from the FakeStore API:
- Product catalog data
- Cart (transaction) data

The API responses are returned in JSON format and contain nested structures that require transformation.


### Fetching carts from FakeStoreAPI

In [4]:
import requests

url = "https://fakestoreapi.com/carts"
response = requests.get(url)
response.status_code

200

In [5]:
len(response.json())

7

In [13]:
carts = response.json()
carts[0]

{'id': 1,
 'userId': 1,
 'date': '2020-03-02T00:00:00.000Z',
 'products': [{'productId': 1, 'quantity': 4},
  {'productId': 2, 'quantity': 1},
  {'productId': 3, 'quantity': 6}],
 '__v': 0}

### Fetching Products

In [17]:
products = requests.get("https://fakestoreapi.com/products").json()

In [18]:
products[0]

{'id': 1,
 'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops',
 'price': 109.95,
 'description': 'Your perfect pack for everyday use and walks in the forest. Stash your laptop (up to 15 inches) in the padded sleeve, your everyday',
 'category': "men's clothing",
 'image': 'https://fakestoreapi.com/img/81fPKd-2AYL._AC_SL1500_t.png',
 'rating': {'rate': 3.9, 'count': 120}}

## 🔹 Transform

The transformation process includes:
- Expanding each cart into multiple line items
- Joining cart items with product data using product IDs
- Calculating revenue at the product level
- Ensuring consistent schema across all rows

This step converts raw API responses into a clean, analysis-ready dataset.

## 🔗 Joining Products with Carts

To calculate revenue, cart data must be joined with product data.

- Cart data provides **product ID and quantity**
- Product data provides **price and category**

A dictionary-based lookup is used to efficiently map:
    
product_id → price, category

In [26]:
product_lookup={
    
    p['id']:{
        
        'category':p['category'],
        'price':p['price']
    }
    for p in products
}

In [27]:
product_lookup[3]["price"]

55.99

In [31]:
rows=[]

for cart in carts:
    
    cart_id=cart['id']
    user_id=cart['userId']
    date=cart['date']
    
    for item in cart['products']:
        
        product_id=item['productId']
        quantity=item['quantity']
        
        price=product_lookup[product_id]['price']
        category=product_lookup[product_id]['category']
        
        revenue=price*quantity
        
        rows.append({
            "cart_id": cart_id,
            "user_id": user_id,
            "date": date,
            "product_id": product_id,
            "category": category,
            "price": price,
            "quantity": quantity,
            "revenue": revenue
        })

In [35]:
df=pd.DataFrame(rows)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["cart_id", "product_id"]).reset_index(drop=True)
df

,cart_id,user_id,date,product_id,category,price,quantity,revenue
0,1,1,2020-03-02 00:00:00+00:00,1,men's clothing,109.95,4,439.80
1,1,1,2020-03-02 00:00:00+00:00,2,men's clothing,22.30,1,22.30
2,1,1,2020-03-02 00:00:00+00:00,3,men's clothing,55.99,6,335.94
3,2,1,2020-01-02 00:00:00+00:00,1,men's clothing,109.95,10,1099.50
4,2,1,2020-01-02 00:00:00+00:00,2,men's clothing,22.30,4,89.20
5,2,1,2020-01-02 00:00:00+00:00,5,jewelery,695.00,2,1390.00
6,3,2,2020-03-01 00:00:00+00:00,1,men's clothing,109.95,2,219.90
7,3,2,2020-03-01 00:00:00+00:00,9,electronics,64.00,1,64.00
8,4,3,2020-01-01 00:00:00+00:00,1,men's clothing,109.95,4,439.80
9,5,3,2020-03-01 00:00:00+00:00,7,jewelery,9.99,1,9.99


### Total Revenue by Categories

In [36]:
df.groupby("category")["revenue"].sum().sort_values(ascending=False)

category
men's clothing      2646.44
jewelery            1410.98
electronics          624.00
women's clothing       9.85
Name: revenue, dtype: float64

### High-performing products

In [37]:
df.groupby("product_id")["revenue"].sum().sort_values(ascending=False)

product_id
1     2199.00
5     1390.00
12     342.00
3      335.94
10     218.00
2      111.50
9       64.00
8       10.99
7        9.99
18       9.85
Name: revenue, dtype: float64

### Average Order Value (AOV)

In [38]:
order_revenue = df.groupby("cart_id")["revenue"].sum()
order_revenue.mean()

670.1814285714285

### Top customers

In [39]:
df.groupby("user_id")["revenue"].sum().sort_values(ascending=False)

user_id
1    3376.74
4     560.00
3     460.78
2     283.90
8       9.85
Name: revenue, dtype: float64

## Loading the Dataset

In [41]:
df.to_csv('Fake Store Dataset.csv', index=False)

print(f"Dataset saved succesfully!!")

Dataset saved succesfully!!
